In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

/home/zf/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:10<00:00,  1.37s/it]


In [2]:
prediction_layer = glm.condgen.lm_head.float().cuda()

In [27]:
import torch
hs = torch.normal(0, 1, [4096]).cuda()
h0 = torch.normal(0, 10, [4096]).cuda()
h1 = hs - h0

In [28]:
prediction = torch.argmax(prediction_layer(hs))
print("Expected prediction: ", prediction.item())

Expected prediction:  9263


In [32]:
try:
    import sys
    del sys.modules['split_llm.glm6b.secure_inference']
    del sys.modules['homomorphic_encryption.bfv']
    print("delete complete!")
except Exception as e:
    print(e)
from split_llm.glm6b.secure_inference import GLM_PredictionProtocol
from split_llm.common.torch_utils import relative_error

delete complete!


In [33]:
from split_llm.common.communication import Communication, Node, SimulatedCommunication
communication = SimulatedCommunication(["n0", "n1", "n2"])
communication.new_stage("Test")

n0 = Node(communication, "n0")
n1 = Node(communication, "n1")
n2 = Node(communication, "n2")

n0.space.final_dense = prediction_layer

prediction_protocol = GLM_PredictionProtocol(n0, n1, n2, 10, device="cuda")
prediction_protocol.prepare()
prediction_protocol.offline_execute()

In [34]:
n0.storage[f"{prediction_protocol.name}:x0"] = h0
n1.storage[f"{prediction_protocol.name}:x1"] = h1
prediction_protocol.online_execute()
print("Computed prediction: ", n1.storage[f"{prediction_protocol.name}:z"])

Computed prediction:  [9263]


In [ ]:
prediction_layer